In [14]:
# Csharp random is dumb, so we have to reimplement it's RNG here
class CSRandom():
    def __init__(self,seed):
        # initialize
        self.MAX_INT = 0x7FFFFFFF
        self.MIN_INT = 0x80000000
        self.seed = seed
        MSEED = 0x09A4EC86
        seedarray = [0]*56

        sub = self.MAX_INT if (seed == self.MIN_INT) else abs(seed)
        mj = MSEED - sub

        seedarray[55] = mj
        mk = 1
        for i in range(1,55):
            ii = (21*i)%55
            seedarray[ii] = mk
            mk = mj - mk
            if mk < 0:
                mk += self.MAX_INT
            mj=seedarray[ii]

        for k in range(1,5):
            for i in range(1,56):
                seedarray[i] -= seedarray[1+(i+30)%55]
                if i == 47:
                    i = 47
                while seedarray[i] < 0:
                    seedarray[i]+=self.MAX_INT
        self._inext = 0
        self._inextp = 21
        self._seed = 1
        self._seedarray = seedarray

    def __sample(self):
        locINext = self._inext
        locINextp = self._inextp

        locINext = 1 if (locINext+1 >= 56) else locINext+1
        locINextp = 1 if (locINextp+1 >= 56) else locINextp+1

        retVal = self._seedarray[locINext]-self._seedarray[locINextp]
        if retVal > self.MAX_INT:
            retVal -= self.MAX_INT
        if retVal == self.MAX_INT: 
            retVal -= 1
        if retVal < 0:
            retVal += self.MAX_INT

        self._seedarray[locINext] = retVal

        self._inext = locINext
        self._inextp = locINextp

        return retVal

    def __sample_lr(self):
        res = self.__sample()
        if self.__sample() % 2 == 0:
                res = -res
        return (res + self.MAX_INT - 1)/(2.0*self.MAX_INT-1)

    def Sample(self):
        return self.__sample()*(1.0/self.MAX_INT)

    def Next(self,minVal=0.5,maxVal=0.5):
        if minVal == 0.5 and maxVal == 0.5:
            return self.__sample()
        elif minVal != 0.5 and maxVal == 0.5:
            return int(self.Sample()*minVal)
        else:
            ran = maxVal - minVal
            if ran <= self.MAX_INT :
                return int(ran*self.Sample()) + minVal
            else:
                return int(ran*self.__sample_lr()) + minVal


In [15]:
mod1diff = lambda x,y: (1+y-x)%1

In [16]:
intercepts = []
offsets = []
a = CSRandom(0)
b = CSRandom(1)
max_count = 10000
idx = 0
while idx < max_count:
    x,y =a.Sample(),b.Sample()
    intercepts.append(x)
    offsets.append(mod1diff(x,y))
    idx += 1
    
seed = 145131
r = CSRandom(seed)
idx = 0
diff = []
while idx < max_count:
    m = r.Sample()
    n = (intercepts[idx] + seed*offsets[idx])%1
    diff.append(m-n)
    idx += 1

In [17]:
max(diff),min(diff)

(5.0357717995552775e-11, -4.6459502911488926e-11)

In [18]:
intercept_str = ','.join(['%.16f' % i for i in intercepts])
offset_str = ','.join(['%.16f' % o for o in offsets])

file_contents = f"""
import numpy as np
class CSRandomPrecomputed:
    _intercepts = np.array([{intercept_str}])
    _offsets = np.array([{offset_str}])
    MAX_INT = 0x7FFFFFFF
    MIN_INT = 0x80000000
    def __init__(self, seed):
        self.seed = abs(seed)
        if self.seed > 0x7FFFFFFF:
            self.seed = self.seed - 0x7FFFFFFF
        self.index = 0

    def Sample(self):
        if self.index >= {len(intercepts)}:
            raise IndexError('CSRandomPrecomputed works for less than {len(intercepts)} consecutive rng calls, use CSRandom if need to maintain long term state')
        val = (self.seed * self._offsets[self.index] + self._intercepts[self.index]) % 1
        self.index += 1
        return val

    def Next(self, minVal=0.5, maxVal=0.5):
        if minVal != 0.5 and maxVal == 0.5:
            if minVal < 0:
                raise ValueError('range must be positive')
            return int(self.Sample() * minVal)
        else:
            ran = maxVal - minVal
            if ran > self.MAX_INT:
                raise ValueError('Cannot handle ranges over negative numbers')
            return int(self.Sample() * ran) + minVal
"""
with open('CSRandomPrecomputed.py','w') as f:
    f.write(file_contents)

In [19]:
from CSRandomPrecomputed import CSRandomPrecomputed

seed = 8199231
a = CSRandomPrecomputed(seed)
b = CSRandom(seed)

idx = 0
try:
    while idx < 10000:
        x = a.Sample()
        y = b.Sample()
        assert abs(x-y) < 1e-8
        idx += 1
except:
    print(f'Failed to match variables: index {idx} -> ({x}) vs ({y}), d: {x-y}')